In [1]:
# Visualization of core TRNs unique to or shared by specific ILC subsets (ILC1, NK, ILC2, CCR6+ILC3, CCR6-ILC3)
# Miraldi*, Pokrovskii* et al. (2018) "Characterization of transcriptional regulatory networks that drive the identities
# and functions of intestinal innate lymphoid cells"

# TO START: In the menu above, choose "Cell" --> "Run All", and network + heatmap will load
# NOTE: Default limits networks to TF-TF edges in top 1 TF / gene model (.93 quantile), to see the full 
# network hit "restore" (in the drop-down menu in cell below) and set threshold to 0 and hit "threshold"
# You can search for gene names in the search box below the network (hit "Match"), and find regulators ("targeted by")
# Change "canvas" to "SVG" (drop-down menu in cell below) to enable drag interactions with nodes & labels
# Change "SVG" to "canvas" to speed up layout operations
# More info about jp_gene_viz and user interface instructions are available on Github: 
# https://github.com/simonsfoundation/jp_gene_viz/blob/master/doc/dNetwork%20widget%20overview.ipynb

# Edge key: 
# Solid lines -- supported by ATAC-seq data and gene expression modeling
# Dotted lines -- supported by gene expression modeling only

In [2]:
# Please choose how many TFs you'd like per network.
# Options are limited to Top: "5", "10", "15", or "Inf" (Inf == no limit) that were significantly enriched per TF
topN = "10"

# directory containing gene expression data and network folder
directory = "."
# folder containing networks
netPath = "qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top" + topN

# coreTF_analysis_sharedUnique_cellstr_heatMap_FDR1E5

# networkInits is a list of tuples (network file, sample to color expression, title of network)
networkInits = [
    ('NK_sp.tsv','NK_SI_3','NK, SI'),
    ('ILC1_NK_sp.tsv','ILC1_SI_3','ILC1 & NK, SI'),
    ('ILC1_sp.tsv','ILC1_SI_3','ILC1, SI'),
    ('CCR6mILC3_sp.tsv','CCR6-ILC3_SI_1','CCR6-ILC3, SI'),
    ('CCR6pILC3_CCR6mILC3_sp.tsv','CCR6+ILC3s_SI_3','ILC3 (CCR6+ & CCR6-), SI'),
    ('CCR6pILC3_sp.tsv','CCR6+ILC3s_SI_3','CCR6+ILC3'),    
    ('ILC2_sp.tsv','ILC2_SI_2','ILC2, SI'),
    ('ILC3_ILC1_NK_combined_sp.tsv','CCR6-ILC3_SI_1','ILC3 and (ILC1 OR NK), SI'),
    ('CCR6mILC3_ILC1_ILC2_sp.tsv','CCR6-ILC3_SI_1','ILC1, ILC2 & CCR6-ILC3, SI'),
    ('CCR6mILC3_ILC1_NK_sp.tsv','CCR6-ILC3_SI_1','ILC1, NK, & CCR6-ILC3, SI'),
    ('CCR6mILC3_ILC1_sp.tsv','CCR6-ILC3_SI_1','ILC1 & CCR6-ILC3, SI'),
    ('CCR6mILC3_ILC2_sp.tsv','CCR6-ILC3_SI_1','ILC2 & CCR6-ILC2, SI'),
    ('CCR6mILC3_NK_sp.tsv','CCR6-ILC3_SI_1','NK & CCR6-ILC3, SI'),    
    ('CCR6pILC3_CCR6mILC3_ILC1_ILC2_sp.tsv','CCR6+ILC3s_SI_3','ILC2 & ILC3, SI'),
    ('CCR6pILC3_CCR6mILC3_ILC1_sp.tsv','CCR6+ILC3s_SI_3','ILC1 & ILC3, SI'),
    ('CCR6pILC3_CCR6mILC3_NK_sp.tsv','CCR6+ILC3s_SI_3','NK & ILC3, SI'),
    ('CCR6pILC3_ILC1_NK_sp.tsv','CCR6+ILC3s_SI_3','CCR6+ILC3, ILC1 & NK, SI'),
    ('CCR6pILC3_ILC1_sp.tsv','CCR6+ILC3s_SI_3','CCR6+ILC3 & ILC1, SI'),
    ('CCR6pILC3_ILC2_sp.tsv','CCR6+ILC3s_SI_3','CCR6+ILC3 & ILC2, SI'),
    ('CCR6pILC3_NK_sp.tsv','CCR6+ILC3s_SI_3','CCR6+ILC3 & NK, SI'),
    ('NK_ILC2_sp.tsv','NK_SI_3','NK & ILC2, SI')]

# name of gene expression file
expressionFile = "GeneExpression/fineILCs_uv156_VSTcounts.txt" #"fineILCs_uv156_VSTcounts_zscore_FDR10_genes_clust.tsv"

# For ease of visualization, limit target genes to cytokines, chemokines, receptors. Hit "Restore" button to see
# all target genes in the TRN.
geneListFile = "GeneSets/chemokineReceptorsKlrs.txt"

# set model size, mean # of TFs / genes, with maximum being ~10 TFs / gene
tfsPerGene = 10

In [3]:
import sys
import os.path
if ".." not in sys.path:
    sys.path.append("..")
from jp_gene_viz import dNetwork
dNetwork.load_javascript_support()
# from jp_gene_viz import multiple_network
from jp_gene_viz import LExpression
LExpression.load_javascript_support()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
for networkInit in networkInits:
    proceed = 1
    currCol = networkInit[1]
    networkFile = networkInit[0]
    fileName = directory + '/' + netPath + '/' + networkFile
    if os.path.isfile(fileName):
        L = LExpression.LinkedExpressionNetwork()
        L.show()
        L.load_network(fileName)
        L.load_heatmap(directory + '/' + expressionFile)
        N = L.network
        N.set_title(networkInit[2])
        threshhold = 1-float(tfsPerGene)/10 # because we're dealing with networks that have mean 10 TFs / gene        
        N.threshhold_slider.value = threshhold
        N.apply_click(None)
        N.draw()
        # Limit genes to target genes of interest and their putative regulators
        geneIn = open(directory + '/' + geneListFile,'r')    
        geneList = list()
        for gene in geneIn:
            geneList.append(gene.strip('\n'))
        geneIn.close()
        N.pattern_text.value = " ".join(geneList)
        N.match_click(None)
        N.targeted_click(None)
        N.layout_dropdown.value = 'fruchterman_reingold'
        N.layout_click()
    #     N.split_click(None)
    #     regulates_click(None)
    #     N.layout_click(None)        
        L.gene_click(None)    

        L.expression.transform_dropdown.value = 'Z score'
        L.expression.apply_transform()
        L.expression.col = currCol    
        L.condition_click(None)
        N.labels_button.value=True



A Jupyter Widget

('Reading network', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/NK_sp.tsv')
('Loading saved layout', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/NK_sp.tsv.layout.json')
Omitting edges, using canvas, and fast layout default because the network is large


A Jupyter Widget

('Reading network', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/ILC1_NK_sp.tsv')
('Loading saved layout', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/ILC1_NK_sp.tsv.layout.json')
Omitting edges, using canvas, and fast layout default because the network is large


A Jupyter Widget

...

('Reading network', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/CCR6pILC3_ILC1_NK_sp.tsv')
('Loading saved layout', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/CCR6pILC3_ILC1_NK_sp.tsv.layout.json')


A Jupyter Widget

('Reading network', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/CCR6pILC3_ILC2_sp.tsv')
('Loading saved layout', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/CCR6pILC3_ILC2_sp.tsv.layout.json')


A Jupyter Widget

('Reading network', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/CCR6pILC3_NK_sp.tsv')
('Loading saved layout', './qATAC_10_1_bias25_maxComb/ILCupDownSetsSI_2fdr1p0E-05_top10/CCR6pILC3_NK_sp.tsv.layout.json')


In [5]:
        N.layout_dropdownvalue = 'fruchterman_reingold'
        N.layout_click(None)
